# Updating My Projects on Modrinth

In [1]:
import json
from os import environ

import requests

Today's the day that Minecraft 1.21.9 releases, and, because I like to get ahead of these things, I've gone ahead and tested all my mods and datapacks against the release candidate, verified which existing versions still work, and created new builds for the projects that didn't. So the last step is going to be _tagging_ these versions as compatible, which is something I can't do in advance, because Modrinth (wisely!) doesn't allow you to tag your projects as compatible with a version that doesn't yet exist.

So usually what happens when the new game drop drops (and Modrinth updates their API) is that I go through their website and tag everything. But while I'm doing that, Modrinth is getting slammed by a thousand modders doing the same thing. So today instead, I'll be doing everything programmatically through [Modrinth's lovely API.](https://docs.modrinth.com/api/)

In [2]:
base_url = "https://api.modrinth.com/v2/"

The two endpoints I'll be using are:
- [`GET /version/{id}`](https://docs.modrinth.com/api/operations/getversion/), and
- [`PATCH /version/{id}`](https://docs.modrinth.com/api/operations/modifyversion/)

To perform these operations (or, at least, `PATCH`), I'll need a Modrinth API token, which I've already encoded as an environment variable.

In [3]:
auth_token = environ["MODRINTH_TOKEN"]
header = {"Authorization": auth_token}

Now let's use the requests library to check on the 1.21.9 version of [one of my projects](https://modrinth.com/datapack/give-it-the-beans). I can get the version ID from the Modrinth website (I _could_ do it [from the API](https://docs.modrinth.com/api/operations/getprojectversions/), but for that I'd need the project ID)

![Where to find the version ID on Modrinth](../_static/modrinth_version_id.png)

In [4]:
version_id = "VosBpsMz"

In [5]:
response = requests.get(base_url + f"version/{version_id}", headers=header)
assert response.status_code == 200
response.text

'{"game_versions":["1.21.6","1.21.7","1.21.8","1.21.9-rc1"],"loaders":["datapack"],"id":"VosBpsMz","project_id":"n6G3LgYm","author_id":"DnpSkHuK","featured":false,"name":"Adding 1.21.9 Compatibility","version_number":"v25.09.25","changelog":"Adds support for 1.21.9","changelog_url":null,"date_published":"2025-09-25T19:11:44.323501Z","downloads":6,"version_type":"release","status":"listed","requested_status":null,"files":[{"hashes":{"sha1":"9078ec032e03fef77742abb717d67d460483325f","sha512":"968b83d61e8cf6f12efce149c087220b3928be2f2eeb8bca4a42aea487794e0f1d695db95843f3de08216f61c5e254041af237aa5b903842c67a478c96168277"},"url":"https://cdn.modrinth.com/data/n6G3LgYm/versions/VosBpsMz/Give%20It%20The%20Beans%20v25.09.25.zip","filename":"Give It The Beans v25.09.25.zip","primary":true,"size":32717,"file_type":null}],"dependencies":[]}'

This text is encoded as a JSON response, so to get the existing list of game versions is simple.

In [6]:
current_versions = json.loads(response.text)["game_versions"]
current_versions

['1.21.6', '1.21.7', '1.21.8', '1.21.9-rc1']

And so to update this list, we can call:

In [7]:
requests.patch(
    base_url + f"version/{version_id}",
    headers=header,
    json={"game_versions": current_versions + ["1.29"]},
)

<Response [400]>

Oh no! But that doesn't work yet.

In [8]:
_.text  # type: ignore

'{"error":"invalid_input","description":"Invalid Input: Provided value \'1.29\' is not a valid variant for game_versions"}'

Because 1.21.9 has yet to drop.

While we wait for the drop to drop, I'll go ahead and grab my list of versions that need tagging and transform the above into a macro.

In [9]:
def tag_compatible(
    version_id: str, game_version: str, rc_version: str | None = None
) -> None:
    """
    Tag a version as compatible with the specified Minecraft version

    Parameters
    ----------
    version_id : str
        The ID of the project version to tag
    game_version: str
        The version of Minecraft to add to the list of compatible tags
    rc_version: str, optional
        If specified, this method will first check to make sure that
        the project version was already tagged as compatible with the
        specified release candidate
    """
    existing_versions = json.loads(
        requests.get(base_url + f"version/{version_id}", headers=header).text
    )["game_versions"]
    assert game_version not in existing_versions, "Already tagged, nothing doing"
    if rc_version and rc_version not in existing_versions:
        raise ValueError(
            f"{version_id} wasn't tagged compatible with {rc_version}."
            " Did you grab the wrong version ID?"
        )
    response = requests.patch(
        base_url + f"version/{version_id}",
        headers=header,
        json={"game_versions": existing_versions + [game_version]},
    )
    if response.status_code != 200:
        raise ValueError(json.loads(response.text))

In [ ]:
for version_id in (
    "VosBpsMz",
    "X1SpHSKP",
    "bQUvQyqn",
    "leuGnsE6",
    "LiRID8kX",
    "5QQyZPhN",
    "Ez3QkH4m",
):
    tag_compatible(version_id, "1.21.9", "1.21.9-rc1")

And there we have it!